In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Prob 1

In [ ]:
def u(x):
    return np.sin((x+np.pi)**2-1)

def finite_diff_d2udx2(f, a, b, k):
    '''
    f = function with input x
    a = start
    b = end
    k = number of grid points
    '''
    n = k - 1    # This is the number of subintervals
    h = 1.*(b-a)/n
    grid = np.linspace(a,b,k)    # Creates the grid with k points, n=k-1 subintervals
    
    one = np.ones(n-1)
    D = -2*np.diag(one,0) + np.diag(one[:-1],1) + np.diag(one[:-1],-1)
    E = np.zeros(n-1)
    E[0] = f(a)
    E[-1] = f(b)
    
    return 1/h**2 * (D.dot(f(grid[1:-1])) + E)

#This uses the alternate method for calculating the second derivative matrix
def finite_diff_alternate_d2udx2(f, a, b, k):
    '''
    f = function with input x
    a = start
    b = end
    k = number of grid points
    '''
    n = k - 1    # This is the number of subintervals
    h = 1.*(b-a)/n
    grid = np.linspace(a,b,k)    # Creates the grid with k points, n=k-1 subintervals
    
    D = np.zeros((n-1,n+1))
    ones = np.ones(n-1)
    D[:,:-2] += np.diag(ones,0)
    D[:,1:-1] += np.diag(-2*ones,0)
    D[:,2:] += np.diag(ones,0)
    
    return 1/h**2 * D.dot(f(grid))

def finite_diff(f,a,b,k):
    '''
    f = function with input x
    a = start
    b = end
    k = number of grid points
    '''
    n = k - 1    # This is the number of subintervals
    h = 1.*(b-a)/n
    grid = np.linspace(a,b,k)    # Creates the grid with k points, n=k-1 subintervals
    
    # Create matrices for d2udx2
    one = np.ones(n-1)
    D = -2*np.diag(one,0) + np.diag(one[:-1],1) + np.diag(one[:-1],-1)
    E = np.zeros(n-1)
    E[0] = f(a)
    E[-1] = f(b)
    
    # Create matrices for dudx
    G = np.diag(np.zeros(n-1),0) + np.diag(one[:-1],1) - np.diag(one[:-1],-1)
    H = np.copy(E)    # Careful to not set H=E since then changing H changes E
    H[0] = -H[0]
    
    d2udx2 = 1./h**2 * (D.dot(f(grid[1:-1])) + E)
    dudx = 1./(2*h) * (G.dot(f(grid[1:-1])) + H)
    
    return d2udx2, dudx

def prob1():
    d2, d = finite_diff(u, 0, 1, 11)
    result = .5*d2 - d
    plt.plot(np.linspace(0,1,11)[1:-1], result)
    plt.show()
    print(result)

prob1()

# Prob 2

In [ ]:
def prob2(epsilon, N):
    '''
    N = number of subintervals
    '''
    def f(x):
        return -1*np.ones_like(x)
    
    alpha = 1
    beta = 3
    
    grid = np.linspace(0,1,N+1)
    h = 1./N
    D = np.diag(-2*epsilon*np.ones(N-1),0) + np.diag((epsilon+h/2.)*np.ones(N-2),-1) + np.diag((epsilon-h/2.)*np.ones(N-2),1)
    D = D/h**2
    b = f(grid[1:-1])
    b[0] -= alpha*(epsilon+h/2)/h**2
    b[-1] -= beta*(epsilon-h/2)/h**2
    
    sol = np.linalg.inv(D).dot(b)
    sol = np.append(alpha,sol)
    sol = np.append(sol,beta)
    
    return sol

plt.plot(np.linspace(0,1,11), prob2(.1,10))
plt.show()

# Problem 3

In [ ]:
def bvp(f, epsilon, alpha, beta, N):
    
    grid = np.linspace(0,1,N+1)
    h = 1./N
    D = np.diag(-2*epsilon*np.ones(N-1),0) + np.diag((epsilon+h/2)*np.ones(N-2),-1) + np.diag((epsilon-h/2)*np.ones(N-2),1)
    D = D/h**2
    b = f(grid[1:-1])
    if isinstance(b, int):
        b *= np.ones(N-1)
    b[0] -= alpha*(epsilon+h/2)/h**2
    b[-1] -= beta*(epsilon-h/2)/h**2
    
    sol = np.linalg.inv(D).dot(b)
    sol = np.append(alpha,sol)
    sol = np.append(sol,beta)
    
    return sol

num_approx = 10 # Number of Approximations
N = 5*np.array([2**j for j in range(num_approx)])
h, max_error = (1.-0)/N[:-1], np.ones(num_approx-1)

# Best numerical solution, used to approximate the true solution.
# bvp returns the grid, and the grid function, approximating the solution
# with N subintervals of equal length.
num_sol_best = bvp(lambda x:-1, epsilon=.1, alpha=1, beta=3, N=N[-1])

for j in range(len(N)-1):
    num_sol = bvp(lambda x:-1, epsilon=.1, alpha=1, beta=3, N=N[j])
    max_error[j] = np.max(np.abs(num_sol - num_sol_best[::2**(num_approx-j-1)] ))
plt.loglog(h,max_error,'.-r',label="$E(h)$")
plt.loglog(h,h**(2.),'-k',label="$h^{\, 2}$")
plt.xlabel("$h$")
plt.legend(loc='best')
plt.show()
print("The order of the finite difference approximation is about ", ( (np.log(max_error[0]) -
np.log(max_error[-1]) )/( np.log(h[0]) - np.log(h[-1]) ) ), ".")

# Problem 4

In [ ]:
def finite_diff_Dirichlet(N, a, b, alpha, beta, a1, a2, a3, f):
    h = 1.*(b-a)/N
    grid = np.linspace(a, b, N+1)
    
    A1 = a1(grid[1:-1])
    if isinstance(A1,float) or isinstance(A1,int):
        A1 *= np.ones(N-1)
    A2 = a2(grid[1:-1])
    if isinstance(A2,float) or isinstance(A2,int):
        A2 *= np.ones(N-1)
    A3 = a3(grid[1:-1])
    if isinstance(A3,float) or isinstance(A3,int):
        A3 *= np.ones(N-1)  
    
    lam_neg = A1/h**2 - A2/(2*h)
    lam_pos = A1/h**2 + A2/(2*h)
    d = -2*A1/h**2 + A3
    
    M = np.zeros((N+1,N+1))
    M[0,0] = 1.
    M[-1,-1] = 1.
    M[1:-1,:-2] += np.diag(lam_neg,0)
    M[1:-1,1:-1] += np.diag(d,0)
    M[1:-1,2:] += np.diag(lam_pos,0)
    
    b = f(grid)
    if isinstance(b,int) or isinstance(b,float):
        b *= np.ones(N+1)
    b[0] = alpha
    b[-1] = beta
    
    sol = np.linalg.inv(M).dot(b)
    
    return grid, sol

A = finite_diff_Dirichlet(10, 0, 1, 1, 3, lambda x:.1, lambda x:-1, lambda x:0, lambda x:-1)

plt.plot(A[0],A[1])
plt.show()

In [ ]:
a1 = lambda x:.1
a2 = lambda x:0
a3 = lambda x:-4*(np.pi-x**2)
f = lambda x:np.cos(x)

A = finite_diff_Dirichlet(1000, 0, np.pi/2, 0, 1, a1, a2, a3, f)
plt.plot(A[0],A[1])
plt.show()

# Problem 5

In [ ]:
eps = [.1, .01, .001]

a1 = lambda x:epsilon
a2 = lambda x:x
a3 = lambda x:0
f = lambda x:-epsilon*np.pi**2*np.cos(np.pi*x)-np.pi*x*np.sin(np.pi*x)

for epsilon in eps:
    A = finite_diff_Dirichlet(1000, -1, 1, -2, 0, a1, a2, a3, f)
    plt.plot(A[0], A[1])

plt.show()

# Problem 6

In [ ]:
eps = [.05, .02]

a1 = lambda x:epsilon+x**2
a2 = lambda x:4*x
a3 = lambda x:2
f = lambda x:0

for epsilon in eps:
    A = finite_diff_Dirichlet(1000, -1, 1, 1./(1+epsilon), 1./(1+epsilon), a1, a2, a3, f)
    plt.plot(A[0],A[1])
    
plt.show()